# Heart Attacks are Funny

This is some analysis to save Rich Evans' life

In [ ]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso#Loading the dataset

## Read in the data

In [ ]:
heart_df = pd.read_csv('heart.csv')

## Show me that sweet data

In [ ]:
heart_df.head(5)

### Features

1. age - age in years
2. sex - sex (1 = male; 0 = female)
3. cp - chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 0 = asymptomatic)
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital)
5. chol - serum cholestoral in mg/dl
6. fbs - fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
7. restecg - resting electrocardiographic results (0 = normal; 1 = having ST-T; 2 = hypertrophy)
8. thalach - maximum heart rate achieved
9. exng - exercise induced angina (1 = yes; 0 = no)
10. oldpeak - ST depression induced by exercise relative to rest
11. slope - the slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping)
12. ca - number of major vessels (0-3) colored by flourosopy
13. thal - 2 = normal; 1 = fixed defect; 3 = reversable defect
14. num - the predicted attribute - diagnosis of heart disease (angiographic disease status) (Value 0 = < diameter narrowing; Value 1 = > 50% diameter narrowing)

Looks like the previous sample had a lot of people who are gonna die. Let's see some live Rich Evans

In [ ]:
good_hearts = heart_df.where(heart_df['output'] == 0).dropna()
good_hearts.head(5)

## Analysis

Let's do some feature selection. Inspired by [this blog](https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b)

In [ ]:
X = heart_df.drop('output', axis=1)   #Feature Matrix
y = heart_df['output']          #Target Variable

### Pearson Correlation

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = heart_df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor['output'])

#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.4]
relevant_features

## Initial thoughts

Chest pain, max heart rate, angia from exercise, and ST depression all mean Rich is gonna die. Makes sense. We better hope he don't got none of that.

One of the assumptions of linear regression is that the independent variables need to be uncorrelated with each other. If these variables are correlated with each other, then we need to keep only one of them and drop the rest. So let us check the correlation of selected features with each other. This can be done either by visually checking it from the above correlation matrix or from the code snippet below

In [ ]:
print(heart_df[["cp","thalachh"]].corr())
print(heart_df[["cp","exng"]].corr())
print(heart_df[["cp","oldpeak"]].corr())
print(heart_df[["exng","thalachh"]].corr())
print(heart_df[["exng","oldpeak"]].corr())
print(heart_df[["thalachh","oldpeak"]].corr())

There are no correlations above 0.5 so we will keep all the features.

### Backward Elimination

Using a simple model (Ordinary Least Squares), feed all features to model. Then iteratively remove worst feature until we reach the desired performance to feature ratio. We will use a features p-value to determine whether it will save Mr. Evans' life.

In [ ]:
#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)

#Fitting sm.OLS model
model = sm.OLS(y, X_1).fit()
model.pvalues

A lower p-value means more statistical significance, so we remove the features with p-value > 0.05.

Now we loop training the model, checking the max p-value, and removing it until there are no features with p-value < 0.5

In [ ]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while(len(cols) > 0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax > 0.05):
        cols.remove(feature_with_p_max)
    else:
        break
        
selected_features_BE = cols
print(selected_features_BE)

Notice sex is in this feature set while not in the feature set which was selected via Pearson Correlation. It is possible that looking at the correlations of all features at once and selecting may give a worse candidate set of features versus knowing the right subset. This 

### RFE

A more sophisticated feature elimination algorithm is the Recursive Feature Elimination (RFE) method which works by recursively removing attributes and building a model on those attributes that remain. It uses accuracy metric to rank the feature according to their importance. The RFE method takes the model to be used and the number of required features as input. It then gives the ranking of all the variables, 1 being most important. It also gives its support, True being relevant feature and False being irrelevant feature.

In [ ]:
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model, 7)

#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)

#Fitting the data to model
model.fit(X_rfe,y)

print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
        
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

The RFE is the most liberal of all the feature selection algorithms so far. It found the best performance using linear regression on a subset of features selected based on the number of features desired was with 11/13 features. Now let's fit the regression model with the number of features suggested and find out which features were selected.

In [ ]:
cols = list(X.columns)
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model, 10)

#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)

#Fitting the data to model
model.fit(X_rfe,y)              
was_selected_array = pd.Series(rfe.support_, index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

### Embedded Methods

This group of algorithms uses regularization methods to select features during training. We'll try with the Ridge regularization.

In [ ]:
reg = RidgeCV()
reg.fit(X, y)
print("Best alpha using built-in RidgeCV: %f" % reg.alpha_)
print("Best score using built-in RidgeCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Ridge Model")

Well children, we regularized Rich Evans' heart. We done good.